# Cleanup - Remove All Demo Objects

This notebook removes everything created by `0_setup.ipynb`:
- Database: UNDERSTOOD_DEMO (and all schemas/tables within)
- Warehouse: UNDERSTOOD_DEMO_WH

**Requirements:** ACCOUNTADMIN role

## Step 1: Use ACCOUNTADMIN Role

In [ ]:
USE ROLE ACCOUNTADMIN;

## Step 2: Remove Event Table Association

Must unset before dropping the database.

In [ ]:
ALTER DATABASE UNDERSTOOD_DEMO UNSET EVENT_TABLE;

## Step 3: Drop Database and Warehouse

In [ ]:
DROP DATABASE IF EXISTS UNDERSTOOD_DEMO CASCADE;
DROP WAREHOUSE IF EXISTS UNDERSTOOD_DEMO_WH;

## Step 4: Verify Cleanup

In [ ]:
SHOW DATABASES LIKE 'UNDERSTOOD_DEMO';
-- Should return 0 rows

In [ ]:
SHOW WAREHOUSES LIKE 'UNDERSTOOD_DEMO_WH';
-- Should return 0 rows

## Cleanup Complete!

All demo objects have been removed. Your account is back to its original state.

# Cleanup - Remove All Demo Objects
**Run this notebook to restore your environment to its original state.**

---

This will remove:
- `UNDERSTOOD_DEMO` database (and all schemas/tables within)
- `UNDERSTOOD_DEMO_WH` warehouse

**⚠️ WARNING:** This is destructive and cannot be undone!

In [ ]:
# ========== CONFIGURATION - MUST MATCH 0_setup.ipynb ==========
DATABASE_NAME = "UNDERSTOOD_DEMO"    # Same as setup
WAREHOUSE_NAME = "UNDERSTOOD_DEMO_WH"  # Same as setup
# ================================================================

## Connect

In [ ]:
from snowflake.snowpark.context import get_active_session

session = get_active_session()
print(f"Connected as: {session.get_current_user()}")
print(f"Role: {session.get_current_role()}")

## Preview What Will Be Deleted

Run this cell first to see what exists:

In [ ]:
print("=== Database ===")
try:
    result = session.sql(f"SHOW SCHEMAS IN DATABASE {DATABASE_NAME}").to_pandas()
    print(f"{DATABASE_NAME} exists with {len(result)} schemas:")
    for schema in result['name'].tolist():
        print(f"  - {schema}")
except:
    print(f"{DATABASE_NAME} does not exist")

print("\n=== Warehouse ===")
try:
    result = session.sql(f"SHOW WAREHOUSES LIKE '{WAREHOUSE_NAME}'").to_pandas()
    if len(result) > 0:
        print(f"{WAREHOUSE_NAME} exists")
    else:
        print(f"{WAREHOUSE_NAME} does not exist")
except:
    print(f"{WAREHOUSE_NAME} does not exist")

## Delete Everything

**Only run this if you're sure you want to remove all demo objects!**

In [ ]:
try:
    session.sql(f"ALTER DATABASE {DATABASE_NAME} UNSET EVENT_TABLE").collect()
    print("Event table association removed")
except Exception as e:
    print(f"Note: {e}")

try:
    session.sql(f"DROP DATABASE IF EXISTS {DATABASE_NAME} CASCADE").collect()
    print(f"Database {DATABASE_NAME} dropped")
except Exception as e:
    print(f"Error dropping database: {e}")

try:
    session.sql(f"DROP WAREHOUSE IF EXISTS {WAREHOUSE_NAME}").collect()
    print(f"Warehouse {WAREHOUSE_NAME} dropped")
except Exception as e:
    print(f"Error dropping warehouse: {e}")

print("\nCleanup complete!")

## Verify Cleanup

In [ ]:
print("Checking if objects still exist...")

db_exists = len(session.sql(f"SHOW DATABASES LIKE '{DATABASE_NAME}'").collect()) > 0
wh_exists = len(session.sql(f"SHOW WAREHOUSES LIKE '{WAREHOUSE_NAME}'").collect()) > 0

if not db_exists and not wh_exists:
    print("All demo objects have been removed!")
else:
    if db_exists:
        print(f"Warning: Database {DATABASE_NAME} still exists")
    if wh_exists:
        print(f"Warning: Warehouse {WAREHOUSE_NAME} still exists")